# Le Post 
IJservicepricipalepost

In [1]:
# 1. Import des bibliothèques nécessaires
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import LocationMode
import pandas as pd
import io
import os


from dotenv import load_dotenv



In [2]:
load_dotenv()


# Configuration Key Vault
key_vault_url = os.getenv('KEY_VAULT_URL')
secret_name = os.getenv('READ_SECRET_NAME')

# Configuration Azure AD
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('READ_CLIENT_ID')

# Configuration Storage
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
container_name = os.getenv('CONTAINER_NAME')


In [3]:
# 3. Fonction pour récupérer le secret depuis Key Vault
def get_secret_from_keyvault():
    try:
        credential = DefaultAzureCredential()
        secret_client = SecretClient(vault_url=key_vault_url, credential=credential)
        secret = secret_client.get_secret(secret_name)
        return secret.value
    except Exception as e:
        print(f"Erreur lors de la récupération du secret: {str(e)}")
        return None

In [4]:

# 4. Création des credentials
def create_credentials():
    client_secret = get_secret_from_keyvault()
    if client_secret:
        return ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )
    return None


In [5]:
# 5. Fonction pour lister les fichiers dans le conteneur
def list_directory_contents():
    try:
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        service_client = DataLakeServiceClient(
            account_url=f"https://{storage_account_name}.dfs.core.windows.net",
            credential=credential
        )
        
        file_system_client = service_client.get_file_system_client(container_name)
        paths = file_system_client.get_paths()
        
        print("Contenu du conteneur:")
        print("-" * 50)
        
        for path in paths:
            if path.is_directory:
                print(f"📁 {path.name} (Dossier)")
            else:
                size_mb = path.content_length / (1024 * 1024)
                print(f"📄 {path.name} ({size_mb:.2f} Mo)")
                
    except Exception as e:
        print(f"Une erreur est survenue: {str(e)}")

## Recuperation d'une liste des fichier dans le data lake 

In [6]:
# 6. Fonction avancée pour lister les fichiers dans le Data Lake
def list_datalake_contents(prefix=None, show_details=True, filter_extension=None):
    try:
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        service_client = DataLakeServiceClient(
            account_url=f"https://{storage_account_name}.dfs.core.windows.net",
            credential=credential
        )
        
        file_system_client = service_client.get_file_system_client(container_name)
        
        # Obtention des chemins avec filtrage optionnel
        if prefix:
            paths = file_system_client.get_paths(path=prefix)
        else:
            paths = file_system_client.get_paths()

        # Création d'une liste pour stocker les informations des fichiers
        files_info = []
        
        print("\n📂 Contenu du Data Lake:")
        print("=" * 80)
        
        total_size = 0
        file_count = 0
        folder_count = 0

        for path in paths:
            # Filtrage par extension si spécifié
            if filter_extension and not path.is_directory:
                if not path.name.lower().endswith(filter_extension.lower()):
                    continue

            if path.is_directory:
                folder_count += 1
                if show_details:
                    print(f"📁 {path.name}/")
                    print(f"   └── Type: Dossier")
                else:
                    print(f"📁 {path.name}/")
            else:
                file_count += 1
                size_mb = path.content_length / (1024 * 1024)
                total_size += size_mb
                
                if show_details:
                    print(f"📄 {path.name}")
                    print(f"   ├── Taille: {size_mb:.2f} Mo")
                    print(f"   ├── Dernière modification: {path.last_modified}")
                    print(f"   └── Type: Fichier")
                else:
                    print(f"📄 {path.name} ({size_mb:.2f} Mo)")

                files_info.append({
                    'name': path.name,
                    'size_mb': size_mb,
                    'last_modified': path.last_modified
                })

        print("\n📊 Résumé:")
        print("=" * 80)
        print(f"Nombre total de dossiers: {folder_count}")
        print(f"Nombre total de fichiers: {file_count}")
        print(f"Taille totale: {total_size:.2f} Mo")
        
        return files_info

    except Exception as e:
        print(f"❌ Une erreur est survenue: {str(e)}")
        return None

# 7. Fonction pour obtenir des statistiques sur les types de fichiers
def get_file_type_statistics(files_info):
    if not files_info:
        return
    
    print("\n📈 Statistiques par type de fichier:")
    print("=" * 80)
    
    # Comptage des extensions
    extension_stats = {}
    for file in files_info:
        extension = file['name'].split('.')[-1].lower() if '.' in file['name'] else 'sans_extension'
        if extension not in extension_stats:
            extension_stats[extension] = {
                'count': 0,
                'total_size': 0
            }
        extension_stats[extension]['count'] += 1
        extension_stats[extension]['total_size'] += file['size_mb']

    # Affichage des statistiques
    for ext, stats in extension_stats.items():
        print(f".{ext}:")
        print(f"   ├── Nombre de fichiers: {stats['count']}")
        print(f"   └── Taille totale: {stats['total_size']:.2f} Mo")

# 8. Exemple d'utilisation
if __name__ == "__main__":
    # Liste tous les fichiers avec détails
    print("\n🔍 Liste détaillée de tous les fichiers:")
    files_info = list_datalake_contents(show_details=True)
    
    # Liste uniquement les fichiers CSV
    print("\n🔍 Liste des fichiers CSV uniquement:")
    csv_files = list_datalake_contents(filter_extension='.csv', show_details=False)
    
    # Affiche les statistiques par type de fichier
    if files_info:
        get_file_type_statistics(files_info)



🔍 Liste détaillée de tous les fichiers:

📂 Contenu du Data Lake:
📁 airbnb_data/
   └── Type: Dossier
📁 airbnb_data/spain/
   └── Type: Dossier
📁 airbnb_data/spain/barcelona/
   └── Type: Dossier
📄 airbnb_data/spain/barcelona/barcelona_reviews.csv
   ├── Taille: 18.48 Mo
   ├── Dernière modification: 2024-12-04 09:03:51
   └── Type: Fichier
📁 airbnb_data/spain/euskadi/
   └── Type: Dossier
📄 airbnb_data/spain/euskadi/euskadi_reviews.csv
   ├── Taille: 81.88 Mo
   ├── Dernière modification: 2024-12-04 09:09:33
   └── Type: Fichier
📁 airbnb_data/spain/girona/
   └── Type: Dossier
📄 airbnb_data/spain/girona/girona_reviews.csv
   ├── Taille: 105.95 Mo
   ├── Dernière modification: 2024-12-04 09:08:19
   └── Type: Fichier
📁 airbnb_data/spain/madrid/
   └── Type: Dossier
📄 airbnb_data/spain/madrid/madrid_reviews.csv
   ├── Taille: 325.81 Mo
   ├── Dernière modification: 2024-12-04 09:09:49
   └── Type: Fichier
📁 airbnb_data/spain/mallorca/
   └── Type: Dossier
📄 airbnb_data/spain/mallorca/ma

Lecture d'un fichier dans le conteneur 

In [7]:
# 6. Fonction pour lire le fichier CSV depuis le Data Lake
def read_csv_from_datalake(file_name):
    try:
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        service_client = DataLakeServiceClient(
            account_url=f"https://{storage_account_name}.dfs.core.windows.net",
            credential=credential
        )
        
        file_system_client = service_client.get_file_system_client(container_name)
        file_client = file_system_client.get_file_client(file_name)
        
        download = file_client.download_file()
        content = download.readall()
        
        df = pd.read_csv(io.BytesIO(content))
        return df
        
    except Exception as e:
        print(f"Une erreur est survenue: {str(e)}")
        return None

# Test de suppression de fichier 

In [8]:
# 7. Fonction pour tester la suppression d'un fichier
def test_delete_file(file_name):
    try:
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        service_client = DataLakeServiceClient(
            account_url=f"https://{storage_account_name}.dfs.core.windows.net",
            credential=credential
        )
        
        file_system_client = service_client.get_file_system_client(container_name)
        file_client = file_system_client.get_file_client(file_name)
        
        print(f"Tentative de suppression du fichier {file_name}...")
        file_client.delete_file()
        print("⚠️ Attention: Le fichier a été supprimé! Vous avez des droits en écriture.")
        
    except Exception as e:
        print("\n🔒 Résultat du test de sécurité:")
        print("-" * 50)
        print(f"Une erreur s'est produite (c'est normal si vous avez uniquement les droits de lecture):")
        print(f"Message d'erreur: {str(e)}")
        
        if "AuthorizationPermissionMismatch" in str(e):
            print("\n✅ Test réussi: Vous avez bien uniquement les droits de lecture!")
        else:
            print("\n❌ Une erreur inattendue s'est produite.")


In [9]:
# 8. Exécution du script
if __name__ == "__main__":
    # Liste des fichiers
    print("\nListe des fichiers dans le conteneur:")
    list_directory_contents()
    
    # Lecture du CSV
    print("\nLecture du fichier CSV:")
    df_reviews = read_csv_from_datalake("reviews.csv")
    
    # Affichage des informations du DataFrame
    if df_reviews is not None:
        print("\nAperçu du fichier reviews.csv:")
        print("-" * 50)
        print("\nInformations sur le DataFrame:")
        print(df_reviews.info())
        print("\nPremières lignes du DataFrame:")
        print(df_reviews.head())
        print("\nDimensions du DataFrame:", df_reviews.shape)
    
    # Test de suppression
    print("\nTest de suppression:")
    test_delete_file("reviews.csv")


Liste des fichiers dans le conteneur:
Contenu du conteneur:
--------------------------------------------------
📁 airbnb_data (Dossier)
📁 airbnb_data/spain (Dossier)
📁 airbnb_data/spain/barcelona (Dossier)
📄 airbnb_data/spain/barcelona/barcelona_reviews.csv (18.48 Mo)
📁 airbnb_data/spain/euskadi (Dossier)
📄 airbnb_data/spain/euskadi/euskadi_reviews.csv (81.88 Mo)
📁 airbnb_data/spain/girona (Dossier)
📄 airbnb_data/spain/girona/girona_reviews.csv (105.95 Mo)
📁 airbnb_data/spain/madrid (Dossier)
📄 airbnb_data/spain/madrid/madrid_reviews.csv (325.81 Mo)
📁 airbnb_data/spain/mallorca (Dossier)
📄 airbnb_data/spain/mallorca/mallorca_reviews.csv (7.79 Mo)
📁 airbnb_data/spain/menorca (Dossier)
📄 airbnb_data/spain/menorca/menorca_reviews.csv (1.40 Mo)
📁 airbnb_data/spain/valencia (Dossier)
📄 airbnb_data/spain/valencia/valencia_reviews.csv (8.64 Mo)
📄 data-00000-of-00105.parquet (439.64 Mo)
📁 huggingface_data (Dossier)
📄 huggingface_data/data-00000-of-00105.parquet (439.64 Mo)
📄 huggingface_data/d